In [1]:
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#training-on-gpu
import torch, torchvision, os, cv2
from torch.utils.data import random_split, Dataset
from torch.nn import Tanh, Linear, ReLU, Sequential, Conv2d, MaxPool2d, Sigmoid, BatchNorm2d, Flatten, ConvTranspose2d
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


class augementData(Dataset):

    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data.shape[0])

    def __getitem__(self, idx):
        item = self.data[idx]
        item = self.transform(item)
        return item

In [2]:
img_dir = "./face_images/"
_, _, files = next(os.walk(img_dir))
dataBGR = []
dataLAB = []
for f1 in files:
    img = cv2.imread(img_dir+f1)
    dataBGR.append(img)
    imageLAB = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    dataLAB.append(imageLAB)
    
dataBGR = np.array(dataBGR, dtype = np.float32) #Change data type into float 32.
dataLAB = np.array(dataLAB, dtype = np.float32) #Change data type into float 32.

In [3]:
# Image data types and what they mean!!!
# https://scikit-image.org/docs/dev/user_guide/data_types.html
# Data type
# Range
# uint8
# 0 to 255
# uint16
# 0 to 65535
# uint32
# 0 to 232 - 1
# float
# -1 to 1 or 0 to 1
# int8
# -128 to 127
# int16
# -32768 to 32767
# int32
# -231 to 231 - 1

#According to openCV documents, the range of L*, B*, C* in openCV are expressed as:
#Articel ource https://rodrigoberriel.com/2014/11/opencv-color-spaces-splitting-channels/#:~:text=The%20Lab%20ranges%20are%3A,(1%20%3E%20L%20%3E%20255)
#0 > L > 100 ⇒ OpenCV range = L*255/100 (1 > L > 255)
#-127 > a > 127 ⇒ OpenCV range = a + 128 (1 > a > 255)
#-127 > b > 127 ⇒ OpenCV range = b + 128 (1 > b > 255)

dataLAB = dataLAB / 255 #Change reange into [0, 1]
dataLAB = np.moveaxis(dataLAB, -1, 1) #Reshape channeL from [B, H, W, C] to [B, C, H, W]
dataLAB = dataLAB[torch.randperm(dataLAB.shape[0],generator=torch.random.manual_seed(42))] #shuffle with random seed 42 to make sure each round with same samples pool.
lengths = [int(len(dataLAB)*0.9), int(len(dataLAB)*0.1)]
trainLAB, testLAB = random_split(dataLAB, lengths ,generator=torch.random.manual_seed(42)) #Shuffle data with random seed 42 before split train and test
trainLAB = np.array(trainLAB)
testLAB = np.array(testLAB)

In [4]:
#%%time
trainLABTensor = torch.tensor(trainLAB)
trainLABTensorx10 = torch.clone(trainLABTensor)
for i in range(9):
    trainLABTensorx10 = torch.cat((trainLABTensorx10, torch.clone(trainLABTensor)), 0)
    
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomResizedCrop((128,128),scale=(0.6, 1.0)),
    torchvision.transforms.ToTensor()
])
trainLABTensorx10Aug = augementData(trainLABTensorx10,transform) #Augement train data.

trainData = [] 
trainLx10 = [] #Input L*
trainAx10 = [] #Matrix a* #Label
trainBx10 = [] #Matrix b* #Label
trainAvg_ax10 = [] #scalar mean a* #Label
trainAvg_bx10 = [] #scalar mean b* #Label
i = 0
for t in trainLABTensorx10Aug:
    meanA = torch.mean(t[1])
    meanB = torch.mean(t[2]) #trainLABTensorx10Aug[i,0,:,:]
    #print(np.array(trainLABTensorx10Aug[i,0,:,:]).shape)
    wh = len(t[0])
    trainData.append([torch.reshape(t[0], (1, wh, wh)), torch.tensor([meanA, meanB]), 
                      torch.tensor([np.array(t[1]), np.array(t[2])])])
    trainLx10.append(t[0])
    trainAx10.append(t[1])
    trainBx10.append(t[2])
    trainAvg_ax10.append(meanA)
    trainAvg_bx10.append(meanB)
    i+=1
trainAvg_ax10, trainAvg_bx10 = np.array(trainAvg_ax10), np.array(trainAvg_bx10)     
trainlabelAvgABx10 = np.stack((trainAvg_ax10, trainAvg_bx10),axis=1)
trainLoader = torch.utils.data.DataLoader(trainData, shuffle=True, batch_size=100)

testDataL = testLAB[:,0,:,:] #Input L
testDataL = testDataL.reshape((testDataL.shape[0],1,testDataL.shape[1],testDataL.shape[2]))
testDataA = testLAB[:,1,:,:] #a* matrix
testDataA = testDataA.reshape((testDataA.shape[0],1,testDataA.shape[1],testDataA.shape[2]))
testDataB = testLAB[:,2,:,:] #b* matrix
testDataB = testDataB.reshape((testDataB.shape[0],1,testDataB.shape[1],testDataB.shape[2]))
testAvg_a = testLAB.mean(axis=(2,3))[:,1] #Get label mean of each a* 
testAvg_b = testLAB.mean(axis=(2,3))[:,2] #Get label mean of each b*
testDataL = torch.tensor(testDataL) #Test input
testlabelAvgAB = np.stack((testAvg_a, testAvg_b),axis=1)
testlabelAvgAB = torch.tensor(testlabelAvgAB) #Test output label

testData = []
for i in range(len(testDataL)):
    testData.append([testDataL[i], testlabelAvgAB[i], torch.tensor([testDataA[i][0], testDataB[i][0]])])
testLoader = torch.utils.data.DataLoader(testData, shuffle=False, batch_size=100)

Wall time: 36.5 s


In [5]:
net = Sequential(
            # Defining 1st 2D convolution layer
            Conv2d(1, 3, kernel_size=3, stride=1, padding=1), #128@3
            BatchNorm2d(3),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 2nd 2D convolution layer
            Conv2d(3, 3, kernel_size=3, stride=1, padding=1), #64@3
            BatchNorm2d(3),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 3rd 2D convolution layer
            Conv2d(3, 3, kernel_size=3, stride=1, padding=1), #32@3
            BatchNorm2d(3),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 4th 2D convolution layer
            Conv2d(3, 3, kernel_size=3, stride=1, padding=1), #16@3
            BatchNorm2d(3),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 5th 2D convolution layer
            Conv2d(3, 3, kernel_size=3, stride=1, padding=1), #8@3
            BatchNorm2d(3),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 6th 2D convolution layer
            Conv2d(3, 3, kernel_size=3, stride=1, padding=1), #4@3
            BatchNorm2d(3),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 7th 2D convolution layer
            Conv2d(3, 3, kernel_size=3, stride=1, padding=1), #2@3
            BatchNorm2d(3),
            ReLU(inplace=True),
            #MaxPool2d(kernel_size=2, stride=2),
            Flatten(),
            Linear(3 * 2 * 2, 2)
        )
net = net.cuda()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

In [6]:
#%%time
trainTestLog = open("RegTrainTestLog1000.txt","w+") 
n_epoch = 1000
for epoch in range(n_epoch):  # loop over the dataset multiple times
#epoch = 0
#while True:
    train_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = data[0].cuda(), data[1].cuda() #Reg
        #inputs, labels = data[0].cuda(), data[2].cuda() #Color
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()
    logStr = 'Train epoch: %d, MSE_Loss: %.10f' % (epoch + 1, train_loss/ (i+1))
    trainTestLog.write(logStr+'\n') #Save training records into log.
    print(logStr)
    epoch += 1

print('Finished Training')

Train epoch: 1, MSE_Loss: 0.0163966381
Train epoch: 2, MSE_Loss: 0.0002780030
Train epoch: 3, MSE_Loss: 0.0002476326
Train epoch: 4, MSE_Loss: 0.0002321774
Train epoch: 5, MSE_Loss: 0.0002237800
Train epoch: 6, MSE_Loss: 0.0002201383
Train epoch: 7, MSE_Loss: 0.0002148526
Train epoch: 8, MSE_Loss: 0.0002089003
Train epoch: 9, MSE_Loss: 0.0002060139
Train epoch: 10, MSE_Loss: 0.0002041849
Train epoch: 11, MSE_Loss: 0.0002015594
Train epoch: 12, MSE_Loss: 0.0002008114
Train epoch: 13, MSE_Loss: 0.0001989846
Train epoch: 14, MSE_Loss: 0.0001992389
Train epoch: 15, MSE_Loss: 0.0001983321
Train epoch: 16, MSE_Loss: 0.0001985289
Train epoch: 17, MSE_Loss: 0.0001937668
Train epoch: 18, MSE_Loss: 0.0001933088
Train epoch: 19, MSE_Loss: 0.0001923591
Train epoch: 20, MSE_Loss: 0.0001918973
Train epoch: 21, MSE_Loss: 0.0001912232
Train epoch: 22, MSE_Loss: 0.0001883845
Train epoch: 23, MSE_Loss: 0.0001885180
Train epoch: 24, MSE_Loss: 0.0001878422
Train epoch: 25, MSE_Loss: 0.0001876751
Train epo

Train epoch: 204, MSE_Loss: 0.0001459961
Train epoch: 205, MSE_Loss: 0.0001446468
Train epoch: 206, MSE_Loss: 0.0001472298
Train epoch: 207, MSE_Loss: 0.0001447733
Train epoch: 208, MSE_Loss: 0.0001434078
Train epoch: 209, MSE_Loss: 0.0001449136
Train epoch: 210, MSE_Loss: 0.0001436171
Train epoch: 211, MSE_Loss: 0.0001447326
Train epoch: 212, MSE_Loss: 0.0001466344
Train epoch: 213, MSE_Loss: 0.0001464179
Train epoch: 214, MSE_Loss: 0.0001432697
Train epoch: 215, MSE_Loss: 0.0001439054
Train epoch: 216, MSE_Loss: 0.0001449756
Train epoch: 217, MSE_Loss: 0.0001436612
Train epoch: 218, MSE_Loss: 0.0001456536
Train epoch: 219, MSE_Loss: 0.0001473161
Train epoch: 220, MSE_Loss: 0.0001448784
Train epoch: 221, MSE_Loss: 0.0001440014
Train epoch: 222, MSE_Loss: 0.0001467398
Train epoch: 223, MSE_Loss: 0.0001438161
Train epoch: 224, MSE_Loss: 0.0001431143
Train epoch: 225, MSE_Loss: 0.0001448918
Train epoch: 226, MSE_Loss: 0.0001437157
Train epoch: 227, MSE_Loss: 0.0001438850
Train epoch: 228

Train epoch: 404, MSE_Loss: 0.0001349531
Train epoch: 405, MSE_Loss: 0.0001349722
Train epoch: 406, MSE_Loss: 0.0001355453
Train epoch: 407, MSE_Loss: 0.0001346323
Train epoch: 408, MSE_Loss: 0.0001346487
Train epoch: 409, MSE_Loss: 0.0001338638
Train epoch: 410, MSE_Loss: 0.0001350068
Train epoch: 411, MSE_Loss: 0.0001346319
Train epoch: 412, MSE_Loss: 0.0001344511
Train epoch: 413, MSE_Loss: 0.0001365894
Train epoch: 414, MSE_Loss: 0.0001356254
Train epoch: 415, MSE_Loss: 0.0001345432
Train epoch: 416, MSE_Loss: 0.0001346580
Train epoch: 417, MSE_Loss: 0.0001345201
Train epoch: 418, MSE_Loss: 0.0001380192
Train epoch: 419, MSE_Loss: 0.0001355555
Train epoch: 420, MSE_Loss: 0.0001349087
Train epoch: 421, MSE_Loss: 0.0001353484
Train epoch: 422, MSE_Loss: 0.0001360539
Train epoch: 423, MSE_Loss: 0.0001347968
Train epoch: 424, MSE_Loss: 0.0001340656
Train epoch: 425, MSE_Loss: 0.0001324095
Train epoch: 426, MSE_Loss: 0.0001341019
Train epoch: 427, MSE_Loss: 0.0001331132
Train epoch: 428

Train epoch: 604, MSE_Loss: 0.0001279418
Train epoch: 605, MSE_Loss: 0.0001289799
Train epoch: 606, MSE_Loss: 0.0001284220
Train epoch: 607, MSE_Loss: 0.0001296166
Train epoch: 608, MSE_Loss: 0.0001277347
Train epoch: 609, MSE_Loss: 0.0001279005
Train epoch: 610, MSE_Loss: 0.0001271817
Train epoch: 611, MSE_Loss: 0.0001300521
Train epoch: 612, MSE_Loss: 0.0001292540
Train epoch: 613, MSE_Loss: 0.0001282424
Train epoch: 614, MSE_Loss: 0.0001268636
Train epoch: 615, MSE_Loss: 0.0001278417
Train epoch: 616, MSE_Loss: 0.0001269359
Train epoch: 617, MSE_Loss: 0.0001275518
Train epoch: 618, MSE_Loss: 0.0001292024
Train epoch: 619, MSE_Loss: 0.0001278075
Train epoch: 620, MSE_Loss: 0.0001291034
Train epoch: 621, MSE_Loss: 0.0001282717
Train epoch: 622, MSE_Loss: 0.0001280418
Train epoch: 623, MSE_Loss: 0.0001266113
Train epoch: 624, MSE_Loss: 0.0001288459
Train epoch: 625, MSE_Loss: 0.0001264935
Train epoch: 626, MSE_Loss: 0.0001279018
Train epoch: 627, MSE_Loss: 0.0001273521
Train epoch: 628

Train epoch: 804, MSE_Loss: 0.0001242554
Train epoch: 805, MSE_Loss: 0.0001223979
Train epoch: 806, MSE_Loss: 0.0001213457
Train epoch: 807, MSE_Loss: 0.0001213532
Train epoch: 808, MSE_Loss: 0.0001225344
Train epoch: 809, MSE_Loss: 0.0001208133
Train epoch: 810, MSE_Loss: 0.0001210360
Train epoch: 811, MSE_Loss: 0.0001215122
Train epoch: 812, MSE_Loss: 0.0001212055
Train epoch: 813, MSE_Loss: 0.0001225083
Train epoch: 814, MSE_Loss: 0.0001217597
Train epoch: 815, MSE_Loss: 0.0001222320
Train epoch: 816, MSE_Loss: 0.0001213628
Train epoch: 817, MSE_Loss: 0.0001222625
Train epoch: 818, MSE_Loss: 0.0001216425
Train epoch: 819, MSE_Loss: 0.0001220874
Train epoch: 820, MSE_Loss: 0.0001216565
Train epoch: 821, MSE_Loss: 0.0001201772
Train epoch: 822, MSE_Loss: 0.0001213894
Train epoch: 823, MSE_Loss: 0.0001216005
Train epoch: 824, MSE_Loss: 0.0001214611
Train epoch: 825, MSE_Loss: 0.0001244738
Train epoch: 826, MSE_Loss: 0.0001215219
Train epoch: 827, MSE_Loss: 0.0001209948
Train epoch: 828

In [7]:
# PATH = './RegWithLR0.1EP1000.pth'
# torch.save(net.state_dict(), PATH)

In [8]:
test_loss = 0
with torch.no_grad():
    for i, data in enumerate(testLoader, 0):
        inputs, labels = data[0].cuda(), data[1].cuda() #Reg
        #inputs, labels = data[0].cuda(), data[2].cuda() #Color
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        # print statistics
        test_loss += loss.item()       
    n_samples = (i+1)*len(outputs)       

logStr = 'Test MSE of the network on the %d test samples: %.10f' % (n_samples, test_loss/(i+1))
trainTestLog.write(logStr+'\n') #Save training records into log.
print(logStr)
trainTestLog.close() 

Test MSE of the network on the 75 test samples: 0.0001194611
